# Lesson 1: Advanced RAG Pipeline

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install trulens_eval "llama-index>=0.9.26" llama-index-embeddings-huggingface 'docarray>=0.39.1' 'protobuf>=4.23.2' 'boto3>=1.33.6'

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

import utils

openai.api_key = utils.get_openai_api_key()


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'>
Doc ID: 62b20483-7c06-47d1-9456-e289fe0706e3
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [6]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [7]:
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [8]:
query_engine = index.as_query_engine()

In [9]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can serve as a stepping stone to larger projects.


## Evaluation setup using TruLens

In [10]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [11]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [12]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [13]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [14]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [15]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]


Pace has a long delay of 39.853593 seconds. There might have been a burst of
requests which may become a problem for the receiver of whatever is being paced.
Consider reducing the `seconds_per_period` (currently 60.0 [seconds]) over which to
maintain pace to reduce burstiness. " Alternatively reduce `marks_per_second`
(currently 1.0 [1/second]) to reduce the number of marks
per second in that period. 



Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]


Pace has a long delay of 36.542648 seconds. There might have been a burst of
requests which may become a problem for the receiver of whatever is being paced.
Consider reducing the `seconds_per_period` (currently 60.0 [seconds]) over which to
maintain pace to reduce burstiness. " Alternatively reduce `marks_per_second`
(currently 1.0 [1/second]) to reduce the number of marks
per second in that period. 



In [16]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [17]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_8f31897d2cbf3e6a440f4105d5f75473,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_8f31897d2cbf3e6a440...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-12T10:15:12.492610"", ""...",2024-04-12T10:15:14.466357,0.8,0.000000,0.9,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",[{'args': {'prompt': 'What are the keys to bui...,1,2057,0.003106
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_a18e2e08d208f8c2431feabede0c3785,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by e...",-,"{""record_id"": ""record_hash_a18e2e08d208f8c2431...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-12T10:15:14.658746"", ""...",2024-04-12T10:15:16.749545,0.9,0.866667,0.1,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'How can teamwork contrib...,2,1701,0.002589
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_46dade840d19cec40e20d553d1b67aef,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it helps indiv...",-,"{""record_id"": ""record_hash_46dade840d19cec40e2...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-12T10:15:16.942488"", ""...",2024-04-12T10:15:18.800462,0.9,0.000000,0.1,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'What is the importance o...,1,1700,0.002587
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_309a4835241b8f8b820160040a4db11c,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_309a4835241b8f8b820...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-12T10:15:18.986831"", ""...",2024-04-12T10:15:20.725988,0.9,1.000000,0.8,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'What are some good habit...,1,1631,0.002465
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_060ac508f2fd19ff56c0fdce25373bbd,"""How can altruism be beneficial in building a ...","""Helping others during one's career journey ca...",-,"{""record_id"": ""record_hash_060ac508f2fd19ff56c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-12T10:15:20.916956"", ""...",2024-04-12T10:15:22.172934,0.8,1.000000,0.5,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'How can altruism be bene...,1,1610,0.002423


In [18]:
# launches on http://localhost:8501/
# in sagemaker go to https://<aws.public.url>/jupyterlab/default/proxy/8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.35:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
# stop dashboard to save memory
tru.stop_dashboard()

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [19]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [20]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [21]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [22]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

Start by identifying AI projects that align with your career goals and are responsible, ethical, and beneficial to people. Consider starting small and gradually increasing the scope, complexity, and impact of your projects over time. Use the steps outlined in the chapters provided to guide you in selecting and scoping AI projects that will contribute to your career development.


In [23]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [24]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

How can teamwork contribute to success in AI?
Teammates play a crucial role in the success of AI projects. Working collaboratively with colleagues who are dedicated, continuously learning, and focused on using AI to benefit all people can significantly impact the outcome of a project. The influence of teammates can shape individual behavior and work ethic, leading to a more productive and successful team dynamic.


Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Is it normal to find parts of AI challenging?
It is normal to find parts of AI challenging.


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

What is the right AI job for me?
The right AI job for you would be one that supports your thriving career, enables you to keep growing, and aligns with your career goals. It is important to research roles and companies online or by talking to friends to find a position that complements your skills and aspirations in the field of artificial intelligence.


In [25]:
tru.get_leaderboard(app_ids=[])

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.718519,0.9,0.44375,1.545455,0.000894


In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

In [ ]:
# stop dashboard to save memory
tru.stop_dashboard()

### 2. Auto-merging retrieval

In [26]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [28]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

Building a portfolio of projects involves progressing from simple to complex undertakings over time. Communication is essential in explaining the value of your work to others and gaining their trust and support. By effectively communicating the potential of your projects, you can engage colleagues, mentors, and managers, who can provide valuable feedback and guidance.


In [29]:
# tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [30]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

What are the keys to building a career in AI?
The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by enabling individuals to work on larger projects that require collaboration and coordination. This allows for the pooling of diverse skills and expertise, leading to more comprehensive solutions and innovative approaches. Effective teamwork also fosters a supportive environment where team members can learn from each other, share ideas, and collectively tackle complex challenges in AI projects.


Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

What is the importance of networking in AI?
Networking in AI is crucial as it allows individuals to build relationships within the community, gain valuable information, and receive referrals to potential employers. It provides opportunities to connect with like-minded professionals, seek advice, and receive support when needed. Building a strong professional network can be instrumental in advancing one's career and accessing new opportunities in the field of AI.


...

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tru.get_leaderboard(app_ids=[])

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

In [ ]:
# stop dashboard to save memory
tru.stop_dashboard()